# ComCamSim 2024 DIA Sprint
Authors: Michael Wood-Vasey <wmwv@pitt.edu>  
Last Verified to Run: 2024-05-31  

DIA Sprint.  This Notebook is to explore and figure out how to calculate the Metrics and visualizations we want to look at dipole orientation across an image and check against other things of interest.  Some key new quantities calculated are

"ip_diffim_DipoleFit_separation"  
"ip_diffim_DipoleFit_orientation"

These are based on the postive and negative lobs of the dipole as fit against the science (pos) and template (neg) images.

Let's see what we can learn about the orientation on the sky.

1. [x] Count dipoles and plot on image
2. [x] Look at pixel stamps of dipoles
3. [x] Spatially plot dipole orientation
4. [x] Plot w.r.t. Parallactic Angle.
5. [x] Plot w.r.t. local astrometric shift from DIA kernel

Run at RSP on USDF.  Run using `d_2024_05_15` on a Large container (because we need the memory).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from lsst.daf.butler import Butler
import lsst.daf.butler as dafButler
from lsst.daf.butler import DatasetNotFoundError
from lsst.pipe.base import Instrument
from lsst.ap.association.transformDiaSourceCatalog import UnpackApdbFlags
import lsst.afw.display as afwDisplay
import lsst.display.astrowidgets
from lsst.ip.diffim.dcrModel import calculateImageParallacticAngle
import lsst.afw.image as afwImage

from astropy.visualization import ZScaleInterval, SqrtStretch, ImageNormalize, MinMaxInterval, LogStretch
from astropy.table import Table, join

afwDisplay.setDefaultBackend('matplotlib')

Key orientation information: repo, collection, dataset types.

In [ ]:
repo = "/repo/embargo"
collection = "u/elhoward/DM-44138/LSSTComCamSim"
collections = [collection, "LSSTComCamSim/templates", "LSSTComCamSim/defaults"]
instrument = "LSSTComCamSim"

Dataset Types we'll be interested in  
`postISRCCD`  (I don't think there's a `calexp` in prompt processing)  
`goodSeeingDiff_differenceExp`  
`goodSeeingDiff_diaSrc`  (or `goodSeeingDiff_diaSrcTable` depending on the schema you want.)

In [ ]:
butler = Butler(repo, collections=collections, instrument=instrument)

In [ ]:
registry = butler.registry
for ref in registry.queryDatasets('goodSeeingDiff_differenceExp'):
    print(ref.dataId)

In [ ]:
data_id = {"detector": 0, "exposure": 7024040400021}
# Some alternate data_ids to look at
# data_id = {"detector": 5, "exposure": 7024040300440}
# data_id = {"detector": 8, "exposure": 7024040200715}
# data_id = {"detector": 2, "exposure": 7024040400358}
data_id["visit"] = data_id["exposure"]

In [ ]:
science = butler.get("initial_pvi", dataId=data_id)
diff = butler.get("goodSeeingDiff_differenceExp", dataId=data_id)
template = butler.get("goodSeeingDiff_templateExp", dataId=data_id)

In [ ]:
# Catalog from raw pipeline output is a `lsst.afw.table.SourceCatalog` and includes 100 blank "sky" sources
full_dia_cat = butler.get("goodSeeingDiff_diaSrc", dataId=data_id)
full_dia_cat = full_dia_cat.asAstropy()

In [ ]:
# Calculate SNR based on Aperture Flux
SNR_THRESHOLD = 5.5

full_dia_cat["snr"] = full_dia_cat["slot_ApFlux_instFlux"] / full_dia_cat["slot_ApFlux_instFluxErr"]

In [ ]:
fit_but_not_dipole = full_dia_cat[full_dia_cat["ip_diffim_DipoleFit_flag"] & ~full_dia_cat["ip_diffim_DipoleFit_flag_classification"] & ~full_dia_cat["sky_source"]]

In [ ]:
dia_sources = full_dia_cat[full_dia_cat["sky_source"] == False]
sky_sources = full_dia_cat[full_dia_cat["sky_source"] == True]

# Deep copy to get a new contiguous catalog
dipoles = dia_sources[dia_sources["ip_diffim_DipoleFit_flag_classification"]]

In [ ]:
PIXEL_FLAG_LIST = [
    'base_PixelFlags_flag_offimage', 'base_PixelFlags_flag_edge', 'base_PixelFlags_flag_interpolated',
    'base_PixelFlags_flag_saturated', 'base_PixelFlags_flag_cr', 'base_PixelFlags_flag_bad',
    'base_PixelFlags_flag_suspect', 'base_PixelFlags_flag_interpolatedCenter', 'base_PixelFlags_flag_saturatedCenter',
    'base_PixelFlags_flag_crCenter', 'base_PixelFlags_flag_suspectCenter'
]
# We are interested in the things that are dipoles because of the shape
# but not things that have obviously bad pixels or are next to the edge
# (or are monitoring "sky sources")
# SHAPE_FLAG_LIST = ["slot_Shape_flag"]

EDGE_FLAG_LIST = ["base_SdssCentroid_flag_edge"]
SKY_SOURCE = ["sky_source"]

FLAG_LIST = PIXEL_FLAG_LIST + EDGE_FLAG_LIST + SKY_SOURCE

In [ ]:
# Calculate SNR based on Aperture Flux
SNR_THRESHOLD = 5.5

def get_good_sources_idx(df, flag_list=FLAG_LIST, snr_threshold=SNR_THRESHOLD):
    bad = np.array(np.zeros(len(df)), dtype=bool)
    for flag in flag_list:
        bad |= df[flag]

    # snr = df["slot_ApFlux_instFlux"] / df["slot_ApFlux_instFluxErr"]
    # Use the already calculated column to avoid calculating a thing in two different places
    snr = df["snr"]
    # This is a diff so take things both above the positive SNR cut and below the negative SNR cut
    good_snr = (snr < -snr_threshold) | (snr > snr_threshold)
        
    good = ~bad
    good &= good_snr
    return good

In [ ]:
good_sources_idx = get_good_sources_idx(dia_sources)
good_sources = dia_sources[good_sources_idx]
good_dipoles = dia_sources[good_sources_idx & dia_sources["ip_diffim_DipoleFit_flag_classification"]]

In [ ]:
plt.scatter(good_dipoles["ip_diffim_DipoleFit_pos_instFlux"], good_dipoles["ip_diffim_DipoleFit_separation"])
plt.xlabel("ip_diffim_DipoleFit_pos_instFlux")
plt.ylabel("ip_diffim_DipoleFit_separation")

In [ ]:
print(f"{len(dia_sources)} total detections.")
print(f"{len(dipoles)} detections classified as dipoles.")
print(f"{len(good_sources)} good detections.")

In [ ]:
psf = science.getPsf()
middle_x, middle_y = 2000, 2000
position = lsst.geom.Point2D(middle_x, middle_y)
sigma = psf.computeShape(position).getDeterminantRadius()
print(2 * sigma)

The `ip_diffim_Dipolefit_separation` is fit to the science and template and not the DIA image.  So we expect separations to be on the order of hundredths of a pixel, not the 2*sigma that the poles are separated by in the difference image.

In [ ]:
separation_threshold = {"all": 0, "reasonable": 0.1, "large": 1.0, "unreasonable": 10}  # pixel
separation_color = {"all": "blue", "small": "gray", "reasonable": "green", "large": "purple", "unreasonable": "red"}
separation_linestyle = {"all": "-", "small": "-", "reasonable": "--", "large": ":", "unreasonable": "-."}
large_separation_threshold = 1  # pixel
unreasonable_separation_threshold = 10  # pixel
small_dipoles = good_dipoles[good_dipoles["ip_diffim_DipoleFit_separation"] < separation_threshold["reasonable"]]
reasonable_dipoles = good_dipoles[(good_dipoles["ip_diffim_DipoleFit_separation"] > separation_threshold["reasonable"]) 
                       & (good_dipoles["ip_diffim_DipoleFit_separation"] <= separation_threshold["large"])]
large_dipoles  = good_dipoles[(good_dipoles["ip_diffim_DipoleFit_separation"] > separation_threshold["large"]) 
                      & (good_dipoles["ip_diffim_DipoleFit_separation"] <= separation_threshold["unreasonable"])]
unbelievably_wide_dipoles = good_dipoles[good_dipoles["ip_diffim_DipoleFit_separation"] > separation_threshold["unreasonable"]]

In [ ]:
# The parallactic angle is in the image metadata
# For an object on the meridian this is degrees CCW of South.
# Range [-180, +180]
# https://github.com/lsst/afw/blob/b3a676b8dc86f8714a928f75276a329461bb25ed/include/lsst/afw/image/VisitInfo.h#L208
par_ang = science.getInfo().getVisitInfo().boresightParAngle.asDegrees()
# Position angle of focal plane +Y direction with respect to North
# https://github.com/lsst/afw/blob/b3a676b8dc86f8714a928f75276a329461bb25ed/include/lsst/afw/image/VisitInfo.h#L170
rot_ang = science.getInfo().getVisitInfo().boresightRotAngle.asDegrees()

# This function from the ip_diffim DCR Model code to angle the orientation of the WCS
# to get the parallactic angle with respect to afwDisplay x, y image
par_angle = calculateImageParallacticAngle(science.getInfo().getVisitInfo(), science.getWcs())

In [ ]:
bins=np.logspace(-2, 2)
plt.hist(good_dipoles["ip_diffim_DipoleFit_separation"], bins=bins, histtype="step", linewidth=2, label="all")
plt.xlabel("Dipole +/- separation [pixels]")
plt.ylabel("#/bin")
plt.xscale("log")
for n, s in separation_threshold.items():
    c = separation_color[n]
    l = separation_linestyle[n]
    plt.axvline(s, color=c, linestyle=l, label=n)

plt.legend();

In [ ]:
bins = np.linspace(-180, +180, 37)
plt.hist(good_dipoles["ip_diffim_DipoleFit_orientation"], bins=bins, label="Dipole Orientation Angle", histtype="step")
plt.hist(reasonable_dipoles["ip_diffim_DipoleFit_orientation"], bins=bins, color=separation_color["reasonable"], label="Reasonable Dipole")
plt.hist(large_dipoles["ip_diffim_DipoleFit_orientation"], bins=bins, color=separation_color["large"], label="Large Dipole")
plt.hist(unbelievably_wide_dipoles["ip_diffim_DipoleFit_orientation"], bins=bins, color=separation_color["unreasonable"], label="Unbelievably Large Dipole")

plt.axvline(((par_angle.asDegrees() + 180) % 360) - 180, color="orange", linestyle="--", linewidth="4", label="Parallactic Angle")
plt.axvline(((par_angle.asDegrees() + 180) % 360), color="orange", linestyle="--", linewidth="2", label="Parallactic Angle + 180")

plt.xlabel("ip_diffim_DipoleFit_orientation [deg]")
plt.ylabel("#/bin")

plt.xlim(-180, +180)
plt.legend()

In [ ]:
par_angle.asDegrees()

In [ ]:
# Plot direction of arrows on sky
Q = plt.quiver(dipoles["base_SdssCentroid_x"],
               dipoles["base_SdssCentroid_y"],
               dipoles["ip_diffim_DipoleFit_separation"],
               dipoles["ip_diffim_DipoleFit_separation"],
               angles=dipoles["ip_diffim_DipoleFit_orientation"],
               scale=2)


Anything more than 10 pixels is spurious.  Let's take a look at them.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

plt.sca(axes[0])
display_template = afwDisplay.Display(frame=fig)
display_template.scale("linear", "zscale")
display_template.image(template)

# The following code
# https://github.com/lsst/ip_diffim/blob/main/python/lsst/ip/diffim/dipoleFitTask.py#L867
# suggests that the ip_diffim_DipoleFit_orientation convention is CCW from +x.
# Plot direction of arrows on sky
plt.quiver(good_dipoles["base_SdssCentroid_x"],
           good_dipoles["base_SdssCentroid_y"],
           good_dipoles["ip_diffim_DipoleFit_separation"],
           good_dipoles["ip_diffim_DipoleFit_separation"],
           angles=good_dipoles["ip_diffim_DipoleFit_orientation"],
           scale=2,
           color=separation_color["all"])

plt.sca(axes[1])
display_mask = afwDisplay.Display(frame=fig)
display_mask.image(template.mask)
# Plot direction of arrows on sky
for df, n in zip([good_dipoles, reasonable_dipoles, large_dipoles, unbelievably_wide_dipoles],
                 ["all", "reasonable", "large", "unreasonable"]):
    c = separation_color[n]
    if c == "gray":
        c = "limegreen"
    plt.quiver(df["base_SdssCentroid_x"],
               df["base_SdssCentroid_y"],
               df["ip_diffim_DipoleFit_separation"],
               df["ip_diffim_DipoleFit_separation"],
               angles=df["ip_diffim_DipoleFit_orientation"],
               scale=2,
               color=c)

center_x, center_y = 2000, 2000
west_to_south = -90  # quiver plots CCW from West.  Par Angle is in terms of CCW from South
plt.quiver(center_x, center_y, 10, 10, angles=par_angle.asDegrees() + west_to_south , color="orange", linestyle="--", linewidth=4);

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 6))
plt.sca(axes[0])
display_diff = afwDisplay.Display(frame=fig)
display_diff.scale("linear", "zscale")
display_diff.mtv(diff)

plt.sca(axes[1])
display_science = afwDisplay.Display(frame=fig)
display_diff.scale("linear", "zscale")
display_diff.mtv(science)

plt.sca(axes[2])
display_template = afwDisplay.Display(frame=fig)
display_template.scale("linear", "zscale")
display_template.mtv(template)

plt.tight_layout()
plt.show()

In [ ]:
STAMP_SIZE = 100
def postage_stamp_from_image(image, x, y, stamp_size=STAMP_SIZE):
    """
    Return postage stamp of stamp_size x stamp_size around given x, y

    x, y can be fractional, but the image will be in original pixels.
    """

    import lsst.geom as geom
    # Naive version that is wrong because it doesn't account for x, y orientation
    center = geom.Point2D(x, y)
    size = geom.Extent2I(stamp_size, stamp_size)
    cutout = image.getCutout(center, size)

    return cutout

In [ ]:
def show_postage_stamps(*args, arrow=None, figsize=(12, 6)):
    n = len(args)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    displays = []
    for i, image in enumerate(args):
        plt.sca(axes[i])
        display = afwDisplay.Display(frame=fig)
        display.setMaskTransparency(80)
        display.scale("linear", "zscale")
        display.mtv(image)

        displays.append(display)

        if arrow is not None:
            plt.quiver(arrow["x"], arrow["y"], arrow["sep"]*np.cos(np.deg2rad(arrow["angle"])), arrow["sep"]*np.sin(np.deg2rad(arrow["angle"])),
                       angles="xy", scale_units="xy", scale=1, color="red")

    plt.tight_layout()
    return display

In [ ]:
def show_dipole_stamps(dipoles):
    """
    Returns the last display in case you want to look at mask plane colors."
    """
    if len(dipoles) < 1:
        print("No dipoles to plot.")
        return

    ids = dipoles["id"]
    x = dipoles["slot_Centroid_x"]
    y = dipoles["slot_Centroid_y"]
    dipole_neg_x = dipoles["ip_diffim_DipoleFit_neg_x"]
    dipole_neg_y = dipoles["ip_diffim_DipoleFit_neg_y"]
    separation = dipoles["ip_diffim_DipoleFit_separation"]
    orientation = dipoles["ip_diffim_DipoleFit_orientation"]
    
    for i, xi, yi, dip_x, dip_y, sep, theta in zip(ids, x, y, dipole_neg_x, dipole_neg_y, separation, orientation):
        # Show image name?
        d = postage_stamp_from_image(diff, xi, yi)
        s = postage_stamp_from_image(science, xi, yi)
        t = postage_stamp_from_image(template, xi, yi)
        arrow = {"x": dip_x, "y": dip_y, "sep": sep, "angle": theta}
        display = show_postage_stamps(d, s, t, arrow=arrow)
        plt.title(i)

    return display

In [ ]:
show_dipole_stamps(unbelievably_wide_dipoles)

In [ ]:
show_dipole_stamps(large_dipoles)

In [ ]:
show_dipole_stamps(reasonable_dipoles)

In [ ]:
# show_dipole_stamps(small_dipoles)

In [ ]:
# good_dipoles.colnames

In [ ]:
dipole_columns_of_interest = [
    "id",
    "slot_ApFlux_instFlux",
    "ip_diffim_DipoleFit_instFlux",
    "ip_diffim_DipoleFit_orientation",
    "ip_diffim_DipoleFit_separation",
    "ip_diffim_DipoleFit_chi2dof",
    "ip_diffim_DipoleFit_nData",
    "ip_diffim_DipoleFit_signalToNoise"]

(good_dipoles[dipole_columns_of_interest]).pprint(max_width=-1, max_lines=-1)

In [ ]:
show_dipole_stamps(good_dipoles)

In [ ]:
x = good_sources["slot_Centroid_x"]
y = good_sources["slot_Centroid_y"]
for xi, yi in zip(x[:10], y[:10]):
    # Show image name?
    d = postage_stamp_from_image(diff, xi, yi)
    s = postage_stamp_from_image(science, xi, yi)
    t = postage_stamp_from_image(template, xi, yi)
    show_postage_stamps(d, s, t)

### Look at the DIA kernel astrometric shift

In [ ]:
# Code based on snippet from RHL

dataId = data_id

def get_dia_astrometric_for_xy(butler, data_id, x, y, npt=10):
    dx = np.empty_like(x.ravel())
    dy = np.empty_like(x.ravel())

    kappa = butler.get("goodSeeingDiff_psfMatchKernel", data_id)
    
    im = afwImage.ImageD(kappa.getDimensions())
  
    hsize = kappa.getWidth()//2
    xy = np.arange(-hsize, hsize+1)

    for i, (_x, _y) in enumerate(zip(x.ravel(), y.ravel())):
        kappa.computeImage(im, False, _x, _y)
        dx[i] = np.average(xy, weights=np.mean(im.array, axis=0))
        dy[i] = np.average(xy, weights=np.mean(im.array, axis=1))
        
    return x, y, dx, dy


def get_dia_astrometric_for_grid(butler, data_id, npt=10):
    w, h = butler.get("goodSeeingDiff_differenceExp" + ".bbox", data_id).getDimensions()
    x = np.empty((npt, npt))
    y = np.empty_like(x)

    for i, _x in enumerate(np.linspace(0, w-1, npt)):
        for j, _y in enumerate(np.linspace(0, h-1, npt)):
            x[i, j] = _x
            y[i, j] = _y

    _, _, dx, dy = get_dia_astrometric_for_xy(butler, data_id, x, y)

    return x, y, dx, dy

In [ ]:
# Grid
x, y, dx, dy = get_dia_astrometric_for_grid(butler, data_id)
# Evaluated at source points
dia_src_x, dia_src_y = good_dipoles["base_SdssCentroid_x"], good_dipoles["base_SdssCentroid_y"]
_, _, dia_src_kernel_dx, dia_src_kernel_dy = get_dia_astrometric_for_xy(butler, data_id, dia_src_x, dia_src_y)
dia_src_kernel_theta = np.arctan2(dia_src_kernel_dy, dia_src_kernel_dx)
dia_src_kernel_separation = np.sqrt(dia_src_kernel_dx**2 + dia_src_kernel_dy**2)


In [ ]:
if len(good_dipoles) > 0:
    _, axes = plt.subplots(1, 2)
    axes[0].hist(np.rad2deg(dia_src_kernel_theta), bins=bins);
    axes[1].hist(dia_src_kernel_separation);

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(8, 8))
SCALE = 0.5

# Plot direction of arrows on sky
Q_dipole = plt.quiver(good_dipoles["base_SdssCentroid_x"],
                      good_dipoles["base_SdssCentroid_y"],
                      good_dipoles["ip_diffim_DipoleFit_separation"],
                      good_dipoles["ip_diffim_DipoleFit_separation"],
                      angles=good_dipoles["ip_diffim_DipoleFit_orientation"],
                      color=separation_color["all"],
                      scale=SCALE,
                      label="Dipole Orientation")

Q_kernel_grid = plt.quiver(x, y, dx, dy,
                           color="gray", scale=SCALE, label="DIA astrometric kernel shift")
Q_kernel_xy = plt.quiver(dia_src_x, dia_src_y, dia_src_kernel_dx, dia_src_kernel_dy,
                         color="black", scale=SCALE, label="DIA astrometric kernel shift")
Q_parallactic = plt.quiver(center_x, center_y, 1, 1, angles=par_angle.asDegrees() + west_to_south ,
                           color="orange", linestyle="--", linewidth=4, label="Parallactic Angle");

length = 20
plt.quiverkey(Q_dipole, 0.1, 1.04, 1e-3*length, f"{length} pixels")
plt.quiverkey(Q_kernel_grid, 0.1, 1.04, 1e-3*length, f"{length} mas")
plt.quiverkey(Q_kernel_xy, 0.1, 1.04, 1e-3*length, f"{length} mas")
plt.quiverkey(Q_parallactic, 0.1, 1.04, 1e-3*length, f"{length} mas")

plt.gca().set_aspect(1);
plt.title("visit: {visit}, detector: {detector}".format(**dataId));

plt.legend()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

for ax in axes:
    ax.axis("off")

ax = plt.subplot(1, 2, 1, projection="polar")

# Make a polar plot of orientation histogram
# Make area the proportional quantity
# https://www.chiark.greenend.org.uk/~peterb/python/polar/index2
# https://matplotlib.org/stable/gallery/pie_and_polar_charts/polar_bar.html#sphx-glr-gallery-pie-and-polar-charts-polar-bar-py

# ax = plt.subplot(projection="polar")

bins = np.linspace(-180, +180, 37)

n, bin_edges = np.histogram(good_dipoles["ip_diffim_DipoleFit_orientation"] - np.rad2deg(dia_src_kernel_theta), bins=bins)

width = bins[1] - bins[0]
bin_centers = bins[:-1] + width/2
bin_centers_rad = np.deg2rad(bin_centers)
width_rad = np.deg2rad(width)

ax.bar(bin_centers_rad, n, width=width_rad, label="ip_diffim_DipoleFit_orientation - dia_kernel_shift")

# Set equal-area plotting
ax.set_yscale("function", functions=(lambda value: np.sqrt(value), lambda radius: radius**2))
ax.set_yticks(np.arange(0, max(n)+1))

plt.legend()

# Make a polar plot of orientation histogram
# Make area the proportional quantity
# https://www.chiark.greenend.org.uk/~peterb/python/polar/index2
# https://matplotlib.org/stable/gallery/pie_and_polar_charts/polar_bar.html#sphx-glr-gallery-pie-and-polar-charts-polar-bar-py

ax = plt.subplot(1, 2, 2, projection="polar")

bins = np.linspace(-180, +180, 37)

dipole_orientation_par_angle_diff = good_dipoles["ip_diffim_DipoleFit_orientation"] - par_angle.asDegrees()
# Remap to [-180, +180]
dipole_orientation_par_angle_diff = (dipole_orientation_par_angle_diff + 180) % 360 - 180

n, bin_edges = np.histogram(dipole_orientation_par_angle_diff, bins=bins)

width = bins[1] - bins[0]
bin_centers = bins[:-1] + width/2
bin_centers_rad = np.deg2rad(bin_centers)
width_rad = np.deg2rad(width)

ax.bar(bin_centers_rad, n, width=width_rad, label="ip_diffim_DipoleFit_orientation - science parallactic angle")

# Set equal-area plotting
ax.set_yscale("function", functions=(lambda value: np.sqrt(value), lambda radius: radius**2))
ax.set_yticks(np.arange(0, max(n)+1))

plt.legend();

Note that it's the difference between the effective DCR direction in the template and the science image that matters here
We plot the science image because we can look that up and it's easily defined.

In [ ]:
plt.scatter(good_dipoles["ip_diffim_DipoleFit_orientation"], np.rad2deg(dia_src_kernel_theta))
plt.xlabel("ip_diffim_DipoleFit_orientation")
plt.ylabel("DIA kernel astrometric shift")
plt.xlim(-180, +180)
plt.ylim(-180, +180)
degree_ticks = [-180, -135, -90, -45, 0, +45, +90, +135, +180]
plt.xticks(degree_ticks)
plt.yticks(degree_ticks)
plt.gca().set_aspect("equal");